In [3]:
import sys
sys.path.append('../')
sys.path.append('/usr/users/fsimone/tejaas')
import numpy as np
import scipy.stats as ss
import random
import os
from iotools.readvcf import ReadVCF


vcf_file="/cbscratch/franco/datasets/gtex_v8/genotypes/vcfs_0.01/GTEX_v8_2019-07-29_WGS_838Indiv_Freeze_NoMissingGT_SNPfilter_MAF0.01_chr22.vcf.gz"
startsnp=0
endsnp=10
fam_file="/cbscratch/franco/datasets/gtex_v8/genotypes/gtex_v8.sample"

vcf = ReadVCF(vcf_file, startsnp, endsnp, samplefile=fam_file)
dosage = vcf.dosage
gt_donor_ids = vcf.donor_ids
snpinfo = vcf.snpinfo

def HWEcheck(x):
    gt = x.tolist()
    f = np.array([0] * 3)
    f[0] = gt.count(0)
    f[1] = gt.count(1)
    f[2] = gt.count(2)
    n = sum(f)
    #p_A = (2 * f[0] + f[1]) / (2 * n)
    #p_a = (2 * f[2] + f[1]) / (2 * n)
    X2 = n * ( (4 * f[0] * f[2] - f[1] ** 2) / ((2 * f[0] + f[1]) * (2 * f[2] + f[1])) )**2
    pval = 1 - ss.chi2.cdf(X2, 1)
    return pval

In [8]:
fam_file_eur="/cbscratch/franco/datasets/gtex_v8/genotypes/gtex_v8_eur.sample"
vcf_eur = ReadVCF(vcf_file, startsnp, endsnp, samplefile=fam_file_eur)
dosage_eur = vcf_eur.dosage
gt_donor_ids_eur = vcf_eur.donor_ids
snpinfo_eur = vcf_eur.snpinfo

In [9]:
i = 0
pval = HWEcheck(dosage[i])
pval_eur = HWEcheck(dosage_eur[i])
print(snpinfo[i].varid, pval, pval_eur)

chr22_11212448_C_T_b38 0.7266266763094025 0.7567150248348234


In [10]:
# wikipedia example of HWE
f = np.array([0] * 3)
f[0] = 1469
f[1] = 138
f[2] = 5
n = sum(f)
X2 = n * ( (4 * f[0] * f[2] - f[1] ** 2) / ((2 * f[0] + f[1]) * (2 * f[2] + f[1])) )**2
pval = 1 - ss.chi2.cdf(X2, 1)
print(X2, pval)

0.8309481891464123 0.3619984934750926


In [ ]:
# Everything is correct, the difference is that most programs nowadays implement the mid-pvalue correction
# https://www.degruyter.com/view/j/sagmb.2013.12.issue-4/sagmb-2012-0039/sagmb-2012-0039.xml
# let's not implement it right now